In [92]:
import sys
sys.path.append('d:\\FRAX_project\\FraxBot\\src\\')

In [93]:
import requests
from pprint import pprint
from pymongo import MongoClient
from time import sleep
# from apscheduler.schedulers.background import BackgroundScheduler, BlockingScheduler
import yaml
import schedule
import numpy as np
import pandas as pd

from common import mongodb_connect

In [94]:
# connect MDB collectioons
(db, telegram_metadata, subscription, pairs, wallet) = mongodb_connect()

Initializing configurations...
Connecting to MongoDB Client...
Getting data collections...


In [95]:
def query_subgraph(query: str):
    print("Inside query_subgraph function!")
    """"Query the subgraph with apost request"""
    request = requests.post('https://api.thegraph.com/subgraphs/name/frax-finance-data/fraxlend-subgraph---mainnet',
                            '',
                            json={'query': query})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception(f"Query failed. HTTP return code is - {request.status_code}")

# Pairs Daily History

## Testing and broken down version of code

In [5]:

query_pairs_dailyhistory = """
{
  pairs {
    address
    name
    symbol
    maxLTV
    
    asset{
      name
      symbol
      decimals
    }
    collateral {
      id
      name
      symbol
      decimals
    }
    dailyHistory(first: 1, orderBy: timestamp, orderDirection: desc) {
      id
      exchangeRate
      totalAssetAmount
      totalAssetShare
      totalCollateral
      totalBorrowAmount
      totalBorrowShare
      interestPerSecond
      
      timestamp
    }
  }
}
"""


In [6]:
results_pairs_dailyhistory = query_subgraph(query_pairs_dailyhistory)
data_pairs_dailyhistory = results_pairs_dailyhistory['data']['pairs']

Inside query_subgraph function!


In [34]:
data_pairs_dailyhistory

[{'address': '0x1fff4a418471a7b44efa023320e02dcdb486ed77',
  'name': 'Fraxlend Interest Bearing FRAX (Curve.fi FRAX/USDC) - 22',
  'symbol': 'fFRAX(crvFRAX)-22',
  'maxLTV': '75000',
  'asset': {'name': 'Frax', 'symbol': 'FRAX', 'decimals': '18'},
  'collateral': {'id': '0x3175df0976dfa876431c2e9ee6bc45b65d3473cc',
   'name': 'Curve.fi FRAX/USDC',
   'symbol': 'crvFRAX',
   'decimals': '18'},
  'dailyHistory': [{'id': '0x1fff4a418471a7b44efa023320e02dcdb486ed77-Recent',
    'exchangeRate': '998736832525489422',
    'totalAssetAmount': '2000200142603777982225657',
    'totalAssetShare': '1999921806885726579143440',
    'totalCollateral': '197000000000000000000',
    'totalBorrowAmount': '140142606537663156265',
    'totalBorrowShare': '140000000000000000000',
    'interestPerSecond': '158269833',
    'timestamp': '1689966827'}]},
 {'address': '0x21e354da5d929a4da55428f6bfd71ed9ffd5001f',
  'name': 'Fraxlend Interest Bearing FRAX (Staked Frax Ether) - 9',
  'symbol': 'fFRAX(sfrxETH)-9',


In [7]:
def simplify_dailyHistory(data_list_dict: list[dict], key: str):
    """
    Converts the daily History key to a dict() from a list(dict())
    """
    for data_dict in data_list_dict:
        # Extract the single dictionary from the list (if present)
        if isinstance(data_dict.get(key), list) and len(data_dict[key]) == 1:
            data_dict[key] = data_dict[key][0]

    return data_list_dict
data_pairs_dailyhistory_2 = simplify_dailyHistory(data_pairs_dailyhistory, "dailyHistory")
# convert to pandas dataframe for faster IR
pdf_pairs_dailyhistory = pd.json_normalize(data_pairs_dailyhistory_2, sep='_')

In [36]:
pdf_pairs_dailyhistory

,address,name,symbol,maxLTV,asset_name,asset_symbol,asset_decimals,collateral_id,collateral_name,collateral_symbol,collateral_decimals,dailyHistory_id,dailyHistory_exchangeRate,dailyHistory_totalAssetAmount,dailyHistory_totalAssetShare,dailyHistory_totalCollateral,dailyHistory_totalBorrowAmount,dailyHistory_totalBorrowShare,dailyHistory_interestPerSecond,dailyHistory_timestamp
0,0x1fff4a418471a7b44efa023320e02dcdb486ed77,Fraxlend Interest Bearing FRAX (Curve.fi FRAX/...,fFRAX(crvFRAX)-22,75000,Frax,FRAX,18,0x3175df0976dfa876431c2e9ee6bc45b65d3473cc,Curve.fi FRAX/USDC,crvFRAX,18,0x1fff4a418471a7b44efa023320e02dcdb486ed77-Recent,998736832525489422,2000200142603777982225657,1999921806885726579143440,197000000000000000000,140142606537663156265,140000000000000000000,158269833,1689966827
1,0x21e354da5d929a4da55428f6bfd71ed9ffd5001f,Fraxlend Interest Bearing FRAX (Staked Frax Et...,fFRAX(sfrxETH)-9,75000,Frax,FRAX,18,0xac3e018457b222d93114458476f3e3416abbe38f,Staked Frax Ether,sfrxETH,18,0x21e354da5d929a4da55428f6bfd71ed9ffd5001f-Recent,504154868524028,530840291805682160970327,450182935034011768418446,335605578112849880474,348218791748544139022864,280543838490334415506910,10287529427,1689966827
2,0x281e6cb341a552e4faccc6b4eef1a6fcc523682d,Fraxlend Interest Bearing FRAX (Curve.fi ETH/f...,fFRAX(frxETHCRV)-21,75000,Frax,FRAX,18,0xf43211935c781d5ca1a41d2041f397b8a7366c7a,Curve.fi ETH/frxETH,frxETHCRV,18,0x281e6cb341a552e4faccc6b4eef1a6fcc523682d-Recent,529737947821753,2000324449856156918599388,1985471196675633628413891,600223036886109094969,632591926173609977924812,631991883846083467465034,261188286,1689966827
3,0x32467a5fc2d72d21e8dce990906547a2b012f382,FraxlendV1 - Wrapped BTC/Frax - Variable Time-...,FraxlendV1 - WBTC/FRAX,75000,Frax,FRAX,18,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,Wrapped BTC,WBTC,8,0x32467a5fc2d72d21e8dce990906547a2b012f382-Recent,3350,5014719367939916217615502,4998429704194062079232111,26705690812,3131298362119387669472760,3115231324299515836244523,79123523,1689966827
4,0x3835a58ca93cdb5f912519ad366826ac9a752510,FraxlendV1 - Curve DAO Token/Frax - Variable T...,FraxlendV1 - CRV/FRAX,75000,Frax,FRAX,18,0xd533a949740bb3306d119cc777fa900ba034cd52,Curve DAO Token,CRV,18,0x3835a58ca93cdb5f912519ad366826ac9a752510-Recent,1271389410455204704,20852687471715732420983980,19562043745436244207644248,55908735238958642254800830,16874302019377199531849887,15455084040142738153509387,2792893210,1689966827
5,0x3a25b9ab8c07ffefee614531c75905e810d8a239,Fraxlend Interest Bearing FRAX (ApeCoin) - 17,fFRAX(APE)-17,75000,Frax,FRAX,18,0x4d224452801aced8b2f0aebe155379bb5d594381,ApeCoin,APE,18,0x3a25b9ab8c07ffefee614531c75905e810d8a239-Recent,458912619697500973,1000508967134911957485129,1000454483679305288980948,449036688995175532395,601060258676752134968,599909077169759979470,158442368,1689966827
6,0x50e627a1df8d665524942ad7ec6392b6ba60293a,FraxlendV1 - Frax Price Index/Frax - Variable ...,fFRAX(FPI),95000,Frax,FRAX,18,0x5ca135cb8527d76e932f34b5145575f9d8cbe08e,Frax Price Index,FPI,18,0x50e627a1df8d665524942ad7ec6392b6ba60293a-Recent,923373210003401569,257193618209593395911,254491384538638133101,37793265365384061970,15705875976956740256,15479764237523850697,79123523,1689966827
7,0x62e3fcdfcbef3fd7ed344aea26939e8980c83cec,Fraxlend Interest Bearing FRAX (Staked Frax Et...,fFRAX(sfrxETH)-8,75000,Frax,FRAX,18,0xac3e018457b222d93114458476f3e3416abbe38f,Staked Frax Ether,sfrxETH,18,0x62e3fcdfcbef3fd7ed344aea26939e8980c83cec-Recent,1983517491217757203866,0,0,0,0,0,0,1689966827
8,0x66bf36dba79d4606039f04b32946a260bcd3ff52,Fraxlend Interest Bearing FRAX (Governance OHM...,fFRAX(gOHM)-11,75000,Frax,FRAX,18,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Governance OHM,gOHM,18,0x66bf36dba79d4606039f04b32946a260bcd3ff52-Recent,352178126529988,4352498231265059941331802,4165862689848313359366035,2201915289072070089637,3617551350904327306631422,3419096873951133814612595,2726073601,1689966827
9,0x74f82bd9d0390a4180daaec92d64cf0708751759,Fraxlend Interest Bea

### Calculate Features

In [16]:
pair_info_dict = data_pairs_dailyhistory_2[3]
pair_info_dict

{'address': '0x32467a5fc2d72d21e8dce990906547a2b012f382',
 'name': 'FraxlendV1 - Wrapped BTC/Frax - Variable Time-Weighted Interest Rate - 1',
 'symbol': 'FraxlendV1 - WBTC/FRAX',
 'maxLTV': '75000',
 'asset': {'name': 'Frax', 'symbol': 'FRAX', 'decimals': '18'},
 'collateral': {'id': '0x2260fac5e5542a773aa44fbcfedf7c193bc2c599',
  'name': 'Wrapped BTC',
  'symbol': 'WBTC',
  'decimals': '8'},
 'dailyHistory': {'id': '0x32467a5fc2d72d21e8dce990906547a2b012f382-Recent',
  'exchangeRate': '3353',
  'totalAssetAmount': '5014707151628134804922340',
  'totalAssetShare': '4998429704194062079232111',
  'totalCollateral': '27279151776',
  'totalBorrowAmount': '3224681691869215602473211',
  'totalBorrowShare': '3208147647584821892401025',
  'interestPerSecond': '79123523',
  'timestamp': '1689918947'}}

In [25]:
pair_asset_decimal = int(pair_info_dict["asset"]["decimals"])
pair_col_decimal = int(pair_info_dict["collateral"]["decimals"])

total_borrow_amt_raw = int(pair_info_dict["dailyHistory"]["totalBorrowAmount"])
total_col_amt_raw = int(pair_info_dict["dailyHistory"]["totalCollateral"])
total_asset_amt_raw = int(pair_info_dict["dailyHistory"]["totalAssetAmount"])

total_borrow_amt_scaled = total_borrow_amt_raw/ 10**pair_asset_decimal
total_col_amt_scaled= total_col_amt_raw/ 10**pair_col_decimal
total_asset_amt_scaled = total_asset_amt_raw/ 10**pair_asset_decimal


pair_asset_decimal, pair_col_decimal, total_borrow_amt_raw, total_col_amt_raw, total_asset_amt_raw, total_asset_amt_scaled

(18,
 8,
 3224681691869215602473211,
 27279151776,
 5014707151628134804922340,
 5014707.151628135)

In [18]:
# calculate exchange rate 
# exchange rate  = 1 /(exchangeRate/ collateral_decimals) # UNIT FRAX
ex_rate_init = int(pair_info_dict["dailyHistory"]["exchangeRate"])/ 10**pair_col_decimal
ex_rate = round(1/ex_rate_init, 5)

ex_rate_init, ex_rate
# show 1 col = this many frqax #A

(3.353e-05, 29824.03817)

In [26]:
# calcualte Borrow APR  & Lend APR 
interest_per_year = round((int(pair_info_dict["dailyHistory"]["interestPerSecond"])/ 10**pair_asset_decimal) * 60 * 60 * 24 * 365, 5)
# borrow_APR = (interestPerSecond / 10**pair_asset_decimal) * * 60 * 60 * 24 * 365 #UNIt - %
borrow_APR = interest_per_year
# lend_APR = (borrow_APR / totalAssetAmount) * totalBorrowAmount #UNIt - %
lend_APR = round((borrow_APR / total_asset_amt_scaled) * total_borrow_amt_scaled, 5)
# borrow_APR > lend_APR
int(pair_info_dict["dailyHistory"]["interestPerSecond"]), borrow_APR , lend_APR

# show percentage *100 -> 0.25% #A

(79123523, 0.0025, 0.00161)

In [27]:
# calculate current LTV
# current LTV = totalBorrowAmount / (totalCollateralAmount * exchange rate)
current_LTV = total_borrow_amt_scaled/ (total_col_amt_scaled * ex_rate)
total_borrow_amt_scaled, total_col_amt_scaled, current_LTV
# round it to - 40% 
# Thi sis total LTV but we also want to know for each user posiiton #A
# user_borrow_amt_scaled / (user_col_amt_scaled * ex_rate(pair level)) # unit is percentage

(3224681.6918692156, 272.79151776, 0.39635974766924426)

In [30]:
# calculate Liquidation price 
# LP = totalBorrowAmount/(totalCollateral * maxLTV) / 10^5 (hard-coded)
max_LTV = int(pair_info_dict["maxLTV"])/ 10**5
liquidation_price = round(total_borrow_amt_scaled / (total_col_amt_scaled * max_LTV), 5)
max_LTV, liquidation_price #Unit FRAX

# show individual user level # dont show pair level 
# user_borrow_amt_scaled / (user_col_amt_scaled * max_LTV

(0.075, 157613.97658)

## Modularized code

In [96]:
query_pairs_dailyhistory = """
{
  pairs {
    address
    name
    symbol
    maxLTV
    
    asset{
      name
      symbol
      decimals
    }
    collateral {
      id
      name
      symbol
      decimals
    }
    dailyHistory(first: 1, orderBy: timestamp, orderDirection: desc) {
      id
      exchangeRate
      totalAssetAmount
      totalAssetShare
      totalCollateral
      totalBorrowAmount
      totalBorrowShare
      interestPerSecond
      
      timestamp
    }
  }
}
"""


In [97]:
def simplify_dailyHistory(data_list_dict: list[dict], key: str):
    """
    Converts the daily History key to a dict() from a list(dict())
    """
    for data_dict in data_list_dict:
        # Extract the single dictionary from the list (if present)
        if isinstance(data_dict.get(key), list) and len(data_dict[key]) == 1:
            data_dict[key] = data_dict[key][0]
        else:
            raise AttributeError(f"Pairs data data {key} format is wrong")

    return data_list_dict


In [98]:
round_decimals = 4

def get_pairs_dailyhistory(query_pairs_dailyhistory):
    """Query the pairs daily histrory and update the data into database and created pandas DF for fater IR"""
    try:
        results_pairs_dailyhistory = query_subgraph(query_pairs_dailyhistory)
    except Exception as error:
        # handle the exception
        print("An exception occurred:", type(error).__name__)
    if results_pairs_dailyhistory['data']:
        if results_pairs_dailyhistory['data']['pairs']:
            data_pairs_dailyhistory = results_pairs_dailyhistory['data']['pairs']
            # returns a list of dictoinary
            # de-list daily history
            data_pairs_dailyhistory = simplify_dailyHistory(data_pairs_dailyhistory, "dailyHistory")
            for pair_info_dict in data_pairs_dailyhistory:
                # Extract decimals
                pair_asset_decimal = int(pair_info_dict["asset"]["decimals"])
                pair_col_decimal = int(pair_info_dict["collateral"]["decimals"])
                assert (pair_asset_decimal!= 0 and pair_col_decimal !=0), "Decimals cant be zero"

                # extract total bprrow/ lend amounts
                total_borrow_amt_raw = int(pair_info_dict["dailyHistory"]["totalBorrowAmount"])
                total_col_amt_raw = int(pair_info_dict["dailyHistory"]["totalCollateral"])
                total_asset_amt_raw = int(pair_info_dict["dailyHistory"]["totalAssetAmount"])
                # calcualte features
                total_borrow_amt_scaled = total_borrow_amt_raw/ 10**pair_asset_decimal
                total_col_amt_scaled= total_col_amt_raw/ 10**pair_col_decimal
                total_asset_amt_scaled = total_asset_amt_raw/ 10**pair_asset_decimal

                pair_info_dict["show_pair_symbol"] = f"{pair_info_dict['collateral']['symbol']}/{pair_info_dict['asset']['symbol']}"
                
                # calculate exchange rate 
                # exchange rate  = 1 /(exchangeRate/ collateral_decimals) # UNIT FRAX
                ex_rate_init = int(pair_info_dict["dailyHistory"]["exchangeRate"])/ 10**pair_col_decimal
                if ex_rate_init == 0:
                    ex_rate_scaled = None
                else:
                    ex_rate_scaled = round(1/ex_rate_init, round_decimals)
                # calcualte Borrow APR  & Lend APR 
                interest_per_year = round((int(pair_info_dict["dailyHistory"]["interestPerSecond"])/ 10**pair_asset_decimal) * 60 * 60 * 24 * 365,
                                        round_decimals)
                # borrow_APR = (interestPerSecond / 10**pair_asset_decimal) * * 60 * 60 * 24 * 365 #UNIt - %
                borrow_APR = interest_per_year
                # lend_APR = (borrow_APR / totalAssetAmount) * totalBorrowAmount #UNIt - %
                if total_asset_amt_scaled ==0:
                    lend_APR = None
                else:
                    lend_APR = round((borrow_APR / total_asset_amt_scaled) * total_borrow_amt_scaled, 5)
                # borrow_APR > lend_APR
                # Add features to the dictionary
                pair_info_dict["dailyHistory"]["total_borrow_amt_scaled"] = total_borrow_amt_scaled
                pair_info_dict["dailyHistory"]["total_col_amt_scaled"] = total_col_amt_scaled
                pair_info_dict["dailyHistory"]["total_asset_amt_scaled"] = total_asset_amt_scaled
                pair_info_dict["dailyHistory"]["ex_rate_scaled"] = ex_rate_scaled
                pair_info_dict["dailyHistory"]["borrow_APR"] = borrow_APR
                pair_info_dict["dailyHistory"]["lend_APR"] = lend_APR

            # convert to pandas dataframe for faster IR
            pdf_pairs_dailyhistory = pd.json_normalize(data_pairs_dailyhistory, sep='_')
    return pdf_pairs_dailyhistory, data_pairs_dailyhistory

In [99]:
pdf_pairs_dailyhistory, data_pairs_dailyhistory = get_pairs_dailyhistory(query_pairs_dailyhistory) 

Inside query_subgraph function!


In [141]:
pdf_pairs_dailyhistory.columns

Index(['address', 'name', 'symbol', 'maxLTV', 'show_pair_symbol', 'asset_name',
       'asset_symbol', 'asset_decimals', 'collateral_id', 'collateral_name',
       'collateral_symbol', 'collateral_decimals', 'dailyHistory_id',
       'dailyHistory_exchangeRate', 'dailyHistory_totalAssetAmount',
       'dailyHistory_totalAssetShare', 'dailyHistory_totalCollateral',
       'dailyHistory_totalBorrowAmount', 'dailyHistory_totalBorrowShare',
       'dailyHistory_interestPerSecond', 'dailyHistory_timestamp',
       'dailyHistory_total_borrow_amt_scaled',
       'dailyHistory_total_col_amt_scaled',
       'dailyHistory_total_asset_amt_scaled', 'dailyHistory_ex_rate_scaled',
       'dailyHistory_borrow_APR', 'dailyHistory_lend_APR'],
      dtype='object')

In [148]:
pdf_pairs_dailyhistory

,address,name,symbol,maxLTV,show_pair_symbol,asset_name,asset_symbol,asset_decimals,collateral_id,collateral_name,...,dailyHistory_totalBorrowAmount,dailyHistory_totalBorrowShare,dailyHistory_interestPerSecond,dailyHistory_timestamp,dailyHistory_total_borrow_amt_scaled,dailyHistory_total_col_amt_scaled,dailyHistory_total_asset_amt_scaled,dailyHistory_ex_rate_scaled,dailyHistory_borrow_APR,dailyHistory_lend_APR
0,0x1fff4a418471a7b44efa023320e02dcdb486ed77,Fraxlend Interest Bearing FRAX (Curve.fi FRAX/...,fFRAX(crvFRAX)-22,75000,crvFRAX/FRAX,Frax,FRAX,18,0x3175df0976dfa876431c2e9ee6bc45b65d3473cc,Curve.fi FRAX/USDC,...,140145313432476353361,140000000000000000000,158269833,1690088939,1.401453e+02,1.970000e+02,2.000200e+06,1.0013,0.0050,0.00000
1,0x21e354da5d929a4da55428f6bfd71ed9ffd5001f,Fraxlend Interest Bearing FRAX (Staked Frax Et...,fFRAX(sfrxETH)-9,75000,sfrxETH/FRAX,Frax,FRAX,18,0xac3e018457b222d93114458476f3e3416abbe38f,Staked Frax Ether,...,348592136585606297619265,280543838490334415506910,10182791065,1690088939,3.485921e+05,3.356056e+02,5.312136e+05,1967.6216,0.3211,0.21071
2,0x281e6cb341a552e4faccc6b4eef1a6fcc523682d,Fraxlend Interest Bearing FRAX (Curve.fi ETH/f...,fFRAX(frxETHCRV)-21,75000,frxETHCRV/FRAX,Frax,FRAX,18,0xf43211935c781d5ca1a41d2041f397b8a7366c7a,Curve.fi ETH/frxETH,...,632612100378983086286072,631991883846083467465034,261188286,1690088939,6.326121e+05,6.002230e+02,2.000345e+06,1870.9049,0.0082,0.00259
3,0x32467a5fc2d72d21e8dce990906547a2b012f382,FraxlendV1 - Wrapped BTC/Frax - Variable Time-...,FraxlendV1 - WBTC/FRAX,75000,WBTC/FRAX,Frax,FRAX,18,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,Wrapped BTC,...,3131328616575705281562024,3115231324299515836244523,79123523,1690088939,3.131329e+06,2.669800e+02,5.014750e+06,29913.2516,0.0025,0.00156
4,0x3835a58ca93cdb5f912519ad366826ac9a752510,FraxlendV1 - Curve DAO Token/Frax - Variable T...,FraxlendV1 - CRV/FRAX,75000,CRV/FRAX,Frax,FRAX,18,0xd533a949740bb3306d119cc777fa900ba034cd52,Curve DAO Token,...,16880057668980635484589787,15455084040142738153509387,2792893210,1690088939,1.688006e+07,5.590874e+07,2.138575e+07,0.7550,0.0881,0.06954
5,0x3a25b9ab8c07ffefee614531c75905e810d8a239,Fraxlend Interest Bearing FRAX (ApeCoin) - 17,fFRAX(APE)-17,75000,APE/FRAX,Frax,FRAX,18,0x4d224452801aced8b2f0aebe155379bb5d594381,ApeCoin,...,601071886665869384377,599909077169759979470,158442368,1690088939,6.010719e+02,4.490367e+02,1.000509e+06,2.0730,0.0050,0.00000
6,0x50e627a1df8d665524942ad7ec6392b6ba60293a,FraxlendV1 - Frax Price Index/Frax - Variable ...,fFRAX(FPI),95000,FPI/FRAX,Frax,FRAX,18,0x5ca135cb8527d76e932f34b5145575f9d8cbe08e,Frax Price Index,...,15706027566452683932,15479764237523850697,79123523,1690088939,1.570603e+01,3.779327e+01,2.571938e+02,1.0831,0.0025,0.00015
7,0x62e3fcdfcbef3fd7ed344aea26939e8980c83cec,Fraxlend Interest Bearing FRAX (Staked Frax Et...,fFRAX(sfrxETH)-8,75000,sfrxETH/FRAX,Frax,FRAX,18,0xac3e018457b222d93114458476f3e3416abbe38f,Staked Frax Ether,...,0,0,0,1690088939,0.000000e+00,0.000000e+00,0.000000e+00,0.0005,0.0000,NaN
8,0x66bf36dba79d4606039f04b32946a260bcd3ff52,Fraxlend Interest Bearing FRAX (Governance OHM...,fFRAX(gOHM)-11,75000,gOHM/FRAX,Frax,FRAX,18,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Governance OHM,...,3618740332324338379811168,3419096873951133814612595,2686556219,1690088939,3.618740e+06,2.209915e+03,4.422221e+06,2860.1026,0.0847,0.06931
9,0x74f82bd9d0390a4180daaec92d64cf0708751759,Fraxlend Interest Bearing FRAX (Frax Price Ind...,fFRAX(FPI)-12,95000,FPI/FRAX,Frax,FRAX,18,0x5ca135cb8527d76e932f34b5145575f9d8cbe08e,Frax Price Index,...,1879421327866024447736016,1841200366724450699960891,124478300,1690088939,1.879421e+06,1.975683e+06,3.096950e+06,1.0831,0.0039,0.00237


In [101]:
data_pairs_dailyhistory[0]

{'address': '0x1fff4a418471a7b44efa023320e02dcdb486ed77',
 'name': 'Fraxlend Interest Bearing FRAX (Curve.fi FRAX/USDC) - 22',
 'symbol': 'fFRAX(crvFRAX)-22',
 'maxLTV': '75000',
 'asset': {'name': 'Frax', 'symbol': 'FRAX', 'decimals': '18'},
 'collateral': {'id': '0x3175df0976dfa876431c2e9ee6bc45b65d3473cc',
  'name': 'Curve.fi FRAX/USDC',
  'symbol': 'crvFRAX',
  'decimals': '18'},
 'dailyHistory': {'id': '0x1fff4a418471a7b44efa023320e02dcdb486ed77-Recent',
  'exchangeRate': '998733069332543128',
  'totalAssetAmount': '2000200145310672795422753',
  'totalAssetShare': '1999921806885726579143440',
  'totalCollateral': '197000000000000000000',
  'totalBorrowAmount': '140145313432476353361',
  'totalBorrowShare': '140000000000000000000',
  'interestPerSecond': '158269833',
  'timestamp': '1690088939',
  'total_borrow_amt_scaled': 140.14531343247634,
  'total_col_amt_scaled': 197.0,
  'total_asset_amt_scaled': 2000200.1453106727,
  'ex_rate_scaled': 1.0013,
  'borrow_APR': 0.005,
  'lend_

# User Positions

## testing and broken down code

In [12]:
query_users_positions = """{
  users(first:100) {
    id
    positions{
      pair{
        id
        symbol
        }
      borrowedAssetShare
      lentAssetShare
      depositedCollateralAmount
      timestamp
    }
  }
}
"""

In [13]:
results_users_positions = query_subgraph(query_users_positions)
data_users_positions = results_users_positions['data']['users']

Inside query_subgraph function!


In [14]:
pprint(data_users_positions[23])

{'id': '0x05995bc5736707208ebdb18bc5be812668b525b2',
 'positions': [{'borrowedAssetShare': '0',
                'depositedCollateralAmount': '0',
                'lentAssetShare': '0',
                'pair': {'id': '0x21e354da5d929a4da55428f6bfd71ed9ffd5001f',
                         'symbol': 'fFRAX(sfrxETH)-9'},
                'timestamp': '1678574267'},
               {'borrowedAssetShare': '40836946489511769186409',
                'depositedCollateralAmount': '39463251184457623427',
                'lentAssetShare': '0',
                'pair': {'id': '0x78bb3aec3d855431bd9289fd98da13f9ebb7ef15',
                         'symbol': 'fFRAX(sfrxETH)-16'},
                'timestamp': '1682232839'}]}


### Calculate Features

In [15]:
user_positions = data_users_positions[23]

positions = user_positions['positions']

positions

[{'pair': {'id': '0x21e354da5d929a4da55428f6bfd71ed9ffd5001f',
   'symbol': 'fFRAX(sfrxETH)-9'},
  'borrowedAssetShare': '0',
  'lentAssetShare': '0',
  'depositedCollateralAmount': '0',
  'timestamp': '1678574267'},
 {'pair': {'id': '0x78bb3aec3d855431bd9289fd98da13f9ebb7ef15',
   'symbol': 'fFRAX(sfrxETH)-16'},
  'borrowedAssetShare': '40836946489511769186409',
  'lentAssetShare': '0',
  'depositedCollateralAmount': '39463251184457623427',
  'timestamp': '1682232839'}]

In [16]:
# TODO: Iterate over all the positions
pos = positions[1]
pos

{'pair': {'id': '0x78bb3aec3d855431bd9289fd98da13f9ebb7ef15',
  'symbol': 'fFRAX(sfrxETH)-16'},
 'borrowedAssetShare': '40836946489511769186409',
 'lentAssetShare': '0',
 'depositedCollateralAmount': '39463251184457623427',
 'timestamp': '1682232839'}

In [46]:
# get the id of each pair
pos_pair_id = pos['pair']['id']
# get the information of the partiular pair
pdf_pair_info = pdf_pairs_dailyhistory[pdf_pairs_dailyhistory['address']==pos_pair_id]

pdf_pair_info

,address,name,symbol,maxLTV,show_pair_symbol,asset_name,asset_symbol,asset_decimals,collateral_id,collateral_name,...,dailyHistory_totalBorrowAmount,dailyHistory_totalBorrowShare,dailyHistory_interestPerSecond,dailyHistory_timestamp,dailyHistory_total_borrow_amt_scaled,dailyHistory_total_col_amt_scaled,dailyHistory_total_asset_amt_scaled,dailyHistory_ex_rate_scaled,dailyHistory_borrow_APR,dailyHistory_lend_APR
10,0x78bb3aec3d855431bd9289fd98da13f9ebb7ef15,Fraxlend Interest Bearing FRAX (Staked Frax Et...,fFRAX(sfrxETH)-16,75000,sfrxETH/FRAX,Frax,FRAX,18,0xac3e018457b222d93114458476f3e3416abbe38f,Staked Frax Ether,...,33580583337038107066797499,33205658505680002148099986,930095377,1690019723,3.358058e+07,39476.055817,4.219714e+07,1992.3816,0.0293,0.02332


In [47]:
round_decimals = 4
pair_asset_decimal = int(pdf_pair_info["asset_decimals"].values[0])

# calcualte borrowed amount
total_b_amt_per_share = int(pdf_pair_info["dailyHistory_totalBorrowAmount"].values[0])/int(pdf_pair_info["dailyHistory_totalBorrowShare"].values[0])
# borrow_amount =((borrowedAssetShare/10** pair_asset_decimal) * (totalBorrowAmount/totalBorrowShare)) # Unit FRAX
user_borrow_amt_scaled = round((int(pos["borrowedAssetShare"])/10**pair_asset_decimal) * total_b_amt_per_share, round_decimals)
total_b_amt_per_share, (int(pos["borrowedAssetShare"])/10**pair_asset_decimal), user_borrow_amt_scaled

(1.0112909922052584, 40836.94648951177, 41298.0361)

In [37]:
# calcualte lent amount
total_l_amt_per_share = int(pdf_pair_info["dailyHistory_totalAssetAmount"].values[0]) / int(pdf_pair_info["dailyHistory_totalAssetShare"].values[0])
# lent_amount =((borrowedAssetShare/10** pair_asset_decimal) * (totalAssetAmount/totalAssetShare)) # Unit FRAX
user_lent_amt_scaled = round((int(pos["lentAssetShare"])/ 10**pair_asset_decimal) * total_l_amt_per_share, round_decimals)
total_l_amt_per_share, user_lent_amt_scaled

(1.0001391738865266, 0.0)

In [43]:
pair_col_decimal = int(pdf_pair_info["collateral_decimals"].values[0])
col_unit = pdf_pairs_dailyhistory["collateral_symbol"].values[0]

# calculate deposited collateral amount #UNIT col_symbol
user_dep_col_amt_scaled = round((int(pos["depositedCollateralAmount"])/ 10**pair_col_decimal) , round_decimals)
user_dep_col_amt_scaled, col_unit

(39.4633, 'crvFRAX')

In [48]:
pdf_pair_info.columns

Index(['address', 'name', 'symbol', 'maxLTV', 'show_pair_symbol', 'asset_name',
       'asset_symbol', 'asset_decimals', 'collateral_id', 'collateral_name',
       'collateral_symbol', 'collateral_decimals', 'dailyHistory_id',
       'dailyHistory_exchangeRate', 'dailyHistory_totalAssetAmount',
       'dailyHistory_totalAssetShare', 'dailyHistory_totalCollateral',
       'dailyHistory_totalBorrowAmount', 'dailyHistory_totalBorrowShare',
       'dailyHistory_interestPerSecond', 'dailyHistory_timestamp',
       'dailyHistory_total_borrow_amt_scaled',
       'dailyHistory_total_col_amt_scaled',
       'dailyHistory_total_asset_amt_scaled', 'dailyHistory_ex_rate_scaled',
       'dailyHistory_borrow_APR', 'dailyHistory_lend_APR'],
      dtype='object')

In [49]:
# calculate current LTV
# user_borrow_amt_scaled / (user_col_amt_scaled * ex_rate(pair level)) # unit is percentage
pair_ex_rate = int(pdf_pair_info["dailyHistory_ex_rate_scaled"].values[0])
total_col_amt_scaled = int(pdf_pair_info["dailyHistory_total_col_amt_scaled"].values[0])
user_current_LTV = user_borrow_amt_scaled/ (total_col_amt_scaled * pair_ex_rate)

pair_ex_rate, total_col_amt_scaled, user_current_LTV

(1992, 39476, 0.0005251784839733846)

In [53]:
# calculate Liquidation price 
# LP = user_borrow_amt_scaled / (user_col_amt_scaled * max_LTV
pair_max_LTV = int(pdf_pair_info["maxLTV"].values[0])/ 10**5

user_liquidation_price_scaled = round(user_borrow_amt_scaled / (user_dep_col_amt_scaled * pair_max_LTV), round_decimals)
pair_max_LTV, user_liquidation_price_scaled #Unit FRAX

# show individual user level # dont show pair level 
# 

(0.75, 1395.3229)

## Modularized code

In [107]:
query_users_positions = """{{
    users({0}) {{
      id
      positions{{
        pair{{
          id
          }}
        borrowedAssetShare
        lentAssetShare
        depositedCollateralAmount
        timestamp
      }}
    }}
  }}
  """

def get_user_positions(query_users_positions: str) -> list[dict]: 
  """Query the grapgh data and get the user positions as list of dictionaries"""
  query_user_chunk_num =1000
  # initially start with the first 200 users
  results_users_positions = query_subgraph(query_users_positions.format(f"first: {query_user_chunk_num}"))
  if results_users_positions['data']:
      if results_users_positions['data']['users']:
          data_users_positions = results_users_positions['data']['users']
  print("initial length of data_users_positions -", len(data_users_positions))
  # then itererate over 200 users and skip

  skip_users = query_user_chunk_num
  while(True):
    print(f"Condition - skip: {skip_users}")
    #TODO: modify this logic when number of users get more than 2000, cuurently skip:200 return only 100 user ids 
    results_users_positions = query_subgraph(query_users_positions.format(f"skip: {skip_users}"))
    skip_users+=query_user_chunk_num
    if results_users_positions['data']:
        if results_users_positions['data']['users']:
            data_users_positions_skip = results_users_positions['data']['users']
            print("length of data_users_positions_skip-", len(data_users_positions_skip))
            if len(data_users_positions_skip)==0:
                break
            else:
                data_users_positions.extend(data_users_positions_skip)
        else:
            break
    else:
        break

  print("total length of data_users_positions -", len(data_users_positions))
  return data_users_positions

In [122]:
data_users_positions = get_user_positions(query_users_positions)

Inside query_subgraph function!
initial length of data_users_positions - 1000
Condition - skip: 1000
Inside query_subgraph function!
length of data_users_positions_skip- 51
Condition - skip: 2000
Inside query_subgraph function!
total length of data_users_positions - 1051


In [109]:
data_users_positions[1]

{'id': '0x0017dfe08bcc0dc9a323ca5d4831e371534e9320',
 'positions': [{'pair': {'id': '0x3835a58ca93cdb5f912519ad366826ac9a752510'},
   'borrowedAssetShare': '0',
   'lentAssetShare': '0',
   'depositedCollateralAmount': '0',
   'timestamp': '1687211711'},
  {'pair': {'id': '0x66bf36dba79d4606039f04b32946a260bcd3ff52'},
   'borrowedAssetShare': '0',
   'lentAssetShare': '0',
   'depositedCollateralAmount': '0',
   'timestamp': '1681698407'},
  {'pair': {'id': '0x74f82bd9d0390a4180daaec92d64cf0708751759'},
   'borrowedAssetShare': '0',
   'lentAssetShare': '0',
   'depositedCollateralAmount': '0',
   'timestamp': '1687229375'},
  {'pair': {'id': '0x78bb3aec3d855431bd9289fd98da13f9ebb7ef15'},
   'borrowedAssetShare': '0',
   'lentAssetShare': '0',
   'depositedCollateralAmount': '6712802086469353',
   'timestamp': '1687191515'},
  {'pair': {'id': '0xa1d100a5bf6bfd2736837c97248853d989a9ed84'},
   'borrowedAssetShare': '0',
   'lentAssetShare': '0',
   'depositedCollateralAmount': '0',
   't

In [113]:
round_decimals = 4
# def create_user_notifications(data_users_positions, pdf_pairs_dailyhistory):
# iterate over the users
for user_pos in data_users_positions:
    # iterate over the different positions 
    if user_pos['positions']:
        for pos in user_pos['positions']:
            break



In [114]:
user_pos,pos

({'id': '0xffcfa5f7b994aeb4c6931e0a6cd95d8984e038ce',
  'positions': [{'pair': {'id': '0x78bb3aec3d855431bd9289fd98da13f9ebb7ef15'},
    'borrowedAssetShare': '14896779844889758011882',
    'lentAssetShare': '0',
    'depositedCollateralAmount': '26620000000000000000',
    'timestamp': '1684971431'}]},
 {'pair': {'id': '0x78bb3aec3d855431bd9289fd98da13f9ebb7ef15'},
  'borrowedAssetShare': '14896779844889758011882',
  'lentAssetShare': '0',
  'depositedCollateralAmount': '26620000000000000000',
  'timestamp': '1684971431'})

In [130]:
def filter_zero_positions(pos: dict) ->bool:
    if abs(int(pos['borrowedAssetShare'])) +  abs(int(pos['lentAssetShare']))+ abs(int(pos['depositedCollateralAmount'])) ==0:
        return False
    else:
        return True

In [142]:
round_decimals = 4
# def create_user_notifications(data_users_positions, pdf_pairs_dailyhistory):
data_notifications = []
# iterate over the users
for user_pos in data_users_positions:
    notif_user_pos = {}
    # add user to new dict
    notif_user_pos = {'id': user_pos['id']}

    # iterate over the different positions
    notif_postions = [] 
    if user_pos['positions']:
        for pos in user_pos['positions']:
            if filter_zero_positions(pos):
                notif_pos= {}
                # get the id of each pair
                pos_pair_id = pos['pair']['id']
                # get the information of the partiular pair
                pdf_pair_info = pdf_pairs_dailyhistory[pdf_pairs_dailyhistory['address']==pos_pair_id]
                # extract pair-level features 
                pair_asset_decimal = int(pdf_pair_info["asset_decimals"].values[0])
                pair_col_decimal = int(pdf_pair_info["collateral_decimals"].values[0])
                col_unit = pdf_pairs_dailyhistory["collateral_symbol"].values[0]
                pair_symbol = pdf_pairs_dailyhistory["show_pair_symbol"].values[0]
                pair_max_LTV = int(pdf_pair_info["maxLTV"].values[0])/ 10**5
                pair_ex_rate = int(pdf_pair_info["dailyHistory_ex_rate_scaled"].values[0])
                total_col_amt_scaled = int(pdf_pair_info["dailyHistory_total_col_amt_scaled"].values[0])
                pair_borrow_APR = int(pdf_pair_info["dailyHistory_borrow_APR"].values[0]) * 100
                pair_lend_APR = int(pdf_pair_info["dailyHistory_lend_APR"].values[0]) * 100
                

                # create a new dictionary
                notif_pos = {'pair_id': pos_pair_id,
                            'pair_symbol': pair_symbol,
                            'collateral_symbol': col_unit,
                            'pair_ex_rate': pair_ex_rate,
                            'pair_borrow_APR': pair_borrow_APR,
                            'pair_lend_APR': pair_lend_APR,}
                
                # calculate features

                # calcualte borrowed amount, shares are not scaled that why take raw features so that the scales cancel out
                try:
                    total_b_amt_per_share = int(pdf_pair_info["dailyHistory_totalBorrowAmount"].values[0])/int(pdf_pair_info["dailyHistory_totalBorrowShare"].values[0])
                    # borrow_amount =((borrowedAssetShare/10** pair_asset_decimal) * (totalBorrowAmount/totalBorrowShare)) # Unit FRAX
                    user_borrow_amt_scaled = round((int(pos["borrowedAssetShare"])/10**pair_asset_decimal) * total_b_amt_per_share, round_decimals)
                except (ZeroDivisionError, TypeError):
                    user_borrow_amt_scaled = None

                # calculate deposited collateral amount #UNIT col_symbol
                user_dep_col_amt_scaled = round((int(pos["depositedCollateralAmount"])/ 10**pair_col_decimal) , round_decimals)

                # calcualte lent amount
                try:
                    total_l_amt_per_share = int(pdf_pair_info["dailyHistory_totalAssetAmount"].values[0]) / int(pdf_pair_info["dailyHistory_totalAssetShare"].values[0])
                    # lent_amount =((borrowedAssetShare/10** pair_asset_decimal) * (totalAssetAmount/totalAssetShare)) # Unit FRAX
                    user_lent_amt_scaled = round((int(pos["lentAssetShare"])/ 10**pair_asset_decimal) * total_l_amt_per_share, round_decimals)
                except (ZeroDivisionError, TypeError):
                    user_lent_amt_scaled = None
                # calculate current LTV
                # user_borrow_amt_scaled / (user_col_amt_scaled * ex_rate(pair level)) # unit is percentage
                try:
                    user_current_LTV = round((user_borrow_amt_scaled/ (total_col_amt_scaled * pair_ex_rate)) *100, 2)
                except (ZeroDivisionError, TypeError):
                    user_current_LTV = None
                # calculate Liquidation price 
                # LP = user_borrow_amt_scaled / (user_col_amt_scaled * max_LTV) #Unit FRAX
                try:
                    user_liquidation_price_scaled = round(user_borrow_amt_scaled / (user_dep_col_amt_scaled * pair_max_LTV), round_decimals)
                except (ZeroDivisionError, TypeError):
                    user_liquidation_price_scaled = None
                notif_pos["user_borrow_amt_scaled"] = user_borrow_amt_scaled
                notif_pos["user_dep_col_amt_scaled"] = user_dep_col_amt_scaled
                notif_pos["user_lent_amt_scaled"] = user_lent_amt_scaled
                notif_pos["user_current_LTV"] = user_current_LTV
                notif_pos["user_liquidation_price_scaled"] = user_liquidation_price_scaled
                notif_postions.append(notif_pos)
                
    notif_user_pos['positions'] = notif_postions
    data_notifications.append(notif_user_pos)
    

            


    
    


In [138]:
data_users_positions[15]

{'id': '0x0430c0cd447397d5411aea19933165bd96c46fe3',
 'positions': [{'pair': {'id': '0x66bf36dba79d4606039f04b32946a260bcd3ff52'},
   'borrowedAssetShare': '221534630420259660740655',
   'lentAssetShare': '0',
   'depositedCollateralAmount': '129490000000000000000',
   'timestamp': '1687288967'}]}

In [149]:
data_notifications[15]

{'id': '0x0430c0cd447397d5411aea19933165bd96c46fe3',
 'positions': [{'pair_id': '0x66bf36dba79d4606039f04b32946a260bcd3ff52',
   'pair_symbol': 'crvFRAX/FRAX',
   'collateral_symbol': 'crvFRAX',
   'pair_ex_rate': 2860,
   'pair_borrow_APR': 0,
   'pair_lend_APR': 0,
   'user_borrow_amt_scaled': 234470.1925,
   'user_dep_col_amt_scaled': 129.49,
   'user_lent_amt_scaled': 0.0,
   'user_current_LTV': 3.71,
   'user_liquidation_price_scaled': 2414.2939}]}